In [1]:
# Import required libraries
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np


In [2]:
# Load CIFAR-10 dataset (contains recyclable-like categories e.g., bottles, cans, etc.)
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Normalize image data to [0, 1]
x_train, x_test = x_train / 255.0, x_test / 255.0

# For a lightweight test, limit the number of samples
x_train, y_train = x_train[:10000], y_train[:10000]
x_test, y_test = x_test[:2000], y_test[:2000]


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [3]:
model = models.Sequential([
    layers.Conv2D(16, (3,3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [4]:
history = model.fit(x_train, y_train, epochs=5, validation_data=(x_test, y_test))


Epoch 1/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 24ms/step - accuracy: 0.2106 - loss: 2.1200 - val_accuracy: 0.4185 - val_loss: 1.6524
Epoch 2/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 11s 27ms/step - accuracy: 0.4343 - loss: 1.5668 - val_accuracy: 0.4600 - val_loss: 1.5250
Epoch 3/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 28ms/step - accuracy: 0.5009 - loss: 1.3982 - val_accuracy: 0.5130 - val_loss: 1.4033
Epoch 4/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 16s 53ms/step - accuracy: 0.5410 - loss: 1.2944 - val_accuracy: 0.5245 - val_loss: 1.3498
Epoch 5/5
313/313 ━━━━━━━━━━━━━━━━━━━━ 15s 36ms/step - accuracy: 0.5688 - loss: 1.2259 - val_accuracy: 0.5385 - val_loss: 1.2944


In [5]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print('\nTest Accuracy:', round(test_acc * 100, 2), '%')


63/63 - 1s - 9ms/step - accuracy: 0.5385 - loss: 1.2944

Test Accuracy: 53.85 %


In [6]:
# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model
with open('recyclable_classifier.tflite', 'wb') as f:
    f.write(tflite_model)


Saved artifact at '/tmp/tmpot_1jfyl'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 32, 32, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  134275375346704: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134275375347856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134275375348240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134275375347280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134275375346896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134275375348432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134275375345360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134275375349584: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [7]:
# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path='recyclable_classifier.tflite')
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Run inference on one image
test_image = np.expand_dims(x_test[0], axis=0).astype(np.float32)
interpreter.set_tensor(input_details[0]['index'], test_image)
interpreter.invoke()
predictions = interpreter.get_tensor(output_details[0]['index'])

print("Predicted class:", np.argmax(predictions))


Predicted class: 3


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)
